In [3]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

df = pd.read_csv('/kaggle/input/genius-song-lyrics-with-language-information/song_lyrics.csv')
df.head()

,title,tag,artist,year,views,features,lyrics,id,language_cld3,language_ft,language
0,Killa Cam,rap,Cam'ron,2004,173166,"{""Cam\\'ron"",""Opera Steve""}","[Chorus: Opera Steve & Cam'ron]\nKilla Cam, Ki...",1,en,en,en
1,Can I Live,rap,JAY-Z,1996,468624,{},"[Produced by Irv Gotti]\n\n[Intro]\nYeah, hah,...",3,en,en,en
2,Forgive Me Father,rap,Fabolous,2003,4743,{},Maybe cause I'm eatin\nAnd these bastards fien...,4,en,en,en
3,Down and Out,rap,Cam'ron,2004,144404,"{""Cam\\'ron"",""Kanye West"",""Syleena Johnson""}",[Produced by Kanye West and Brian Miller]\n\n[...,5,en,en,en
4,Fly In,rap,Lil Wayne,2005,78271,{},"[Intro]\nSo they ask me\n""Young boy\nWhat you ...",6,en,en,en


In [50]:
# drop the artist, year, features, id, language_cld3, language_ft columns
df2 = df.drop(['artist', 'year', 'features', 'id', 'language_cld3', 'language_ft'], axis=1)
df2.shape

(5134856, 5)

In [51]:
# drop all rows that arent in english
df2 = df2[df2['language'] == 'en']
df2 = df2.drop(['language'], axis=1)
df2.columns

Index(['title', 'tag', 'views', 'lyrics'], dtype='object')

In [52]:
# keep the 2000 most popular songs from each genre
df3 = df2.groupby('tag').apply(lambda x: x.sort_values('views', ascending=False).head(2000)).reset_index(drop=True)
df3.groupby('tag').size()

tag
country    2000
misc       2000
pop        2000
rap        2000
rb         2000
rock       2000
dtype: int64

In [53]:
# remove all songs with the genre 'misc'
df3 = df3.drop(['views'], axis=1)
df3 = df3[df3['tag'] != 'misc']
df3.groupby('tag').size()

tag
country    2000
pop        2000
rap        2000
rb         2000
rock       2000
dtype: int64

In [103]:
df3.shape

(10000, 3)

In [106]:
# drop all songs with more than 1100 words
df3 = df3[df3['lyrics'].str.split().str.len().lt(1100)]
df3.shape

(9874, 3)

In [107]:
# split data into a train and validation set with pandas and a 97/3 split
train = df3.sample(frac=0.97, random_state=42)
val = df3.drop(train.index)
train.shape, val.shape

((9578, 3), (296, 3))

In [108]:
# save train and validation sets to csv files
train.to_csv('train.csv', index=False)
val.to_csv('val.csv', index=False)